In [ ]:
# Install the library (run this cell if using Colab or if you haven't installed the package)
!pip install simple-backtest yfinance

# Advanced Parameter Optimization

This notebook covers advanced optimization techniques:

1. **Grid Search** - Exhaustive search of parameter space
2. **Random Search** - Efficient sampling for large spaces
3. **Walk-Forward Optimization** - Prevent overfitting with train/test splits
4. **Custom Optimizers** - Creating your own optimization methods
5. **Best Practices** - Avoiding common pitfalls

Parameter optimization is crucial for maximizing strategy performance!

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from typing import Any, Dict, List, Type
import time

from simple_backtest import BacktestConfig, Backtest
from simple_backtest.strategy import Strategy, MovingAverageStrategy
from simple_backtest.optimization import GridSearchOptimizer, RandomSearchOptimizer, WalkForwardOptimizer, Optimizer
from simple_backtest.visualization import plot_equity_curve

## Load Data

In [2]:
# Download data - larger time period for optimization
ticker = "SPY"
data = yf.download(ticker, start="2015-01-01", end="2023-12-31", progress=False)

# Handle MultiIndex columns if present
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

data = data.dropna()

print(f"Data shape: {data.shape}")
print(f"Date range: {data.index[0]} to {data.index[-1]}")

/var/folders/9y/pp82h2912h3781myzdncvs_r0000gp/T/ipykernel_55236/3130999582.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2015-01-01", end="2023-12-31", progress=False)


Data shape: (2264, 5)
Date range: 2015-01-02 00:00:00 to 2023-12-29 00:00:00


## 1. Grid Search Optimization

**Grid Search** tests every possible parameter combination.

**Pros:**
- Guaranteed to find best combination in search space
- Systematic and comprehensive

**Cons:**
- Computationally expensive for large spaces
- Time grows exponentially with parameters

In [3]:
# Configure backtest
config = BacktestConfig(
    initial_capital=10000.0,
    lookback_period=50,
    commission_type="percentage",
    commission_value=0.001,  # 0.1% per trade
    execution_price="open",
    risk_free_rate=0.02,
)

print("Backtest Configuration:")
print(f"  Initial Capital: ${config.initial_capital:,.2f}")
print(f"  Commission: {config.commission_value*100}% per trade")
print(f"  Lookback Period: {config.lookback_period} days")

Backtest Configuration:
  Initial Capital: $10,000.00
  Commission: 0.1% per trade
  Lookback Period: 50 days


In [4]:
# Define parameter space
param_space = {
    'short_window': [5, 10, 15, 20],
    'long_window': [30, 40, 50, 60],
    'shares': [10]  # Keep constant
}

# Calculate total combinations
total_combinations = 1
for values in param_space.values():
    total_combinations *= len(values)

print("Grid Search Configuration:")
for param, values in param_space.items():
    print(f"  {param}: {values}")
print(f"\nTotal combinations: {total_combinations}")

Grid Search Configuration:
  short_window: [5, 10, 15, 20]
  long_window: [30, 40, 50, 60]
  shares: [10]

Total combinations: 16


In [5]:
# Run grid search
grid_optimizer = GridSearchOptimizer(verbose=True)

start_time = time.time()
grid_results = grid_optimizer.optimize(
    data=data,
    config=config,
    strategy_class=MovingAverageStrategy,
    param_space=param_space,
    metric='sharpe_ratio'
)
grid_time = time.time() - start_time

print(f"\n⏱️  Grid search completed in {grid_time:.2f} seconds")

Testing 16 parameter combinations...


Grid Search: 100%|██████████| 16/16 [00:02<00:00,  5.64it/s]


⏱️  Grid search completed in 2.84 seconds


In [6]:
# Display top results
print("\nTOP 10 PARAMETER COMBINATIONS (Grid Search)")
print("=" * 90)
display_cols = ['short_window', 'long_window', 'sharpe_ratio', 'total_return', 'max_drawdown', 'total_trades']
print(grid_results[display_cols].head(10).to_string(index=False))
print("=" * 90)


TOP 10 PARAMETER COMBINATIONS (Grid Search)
 short_window  long_window  sharpe_ratio  total_return  max_drawdown  total_trades
            5           60      0.000000      0.000000      0.000000             0
           10           60      0.000000      0.000000      0.000000             0
           15           60      0.000000      0.000000      0.000000             0
           20           60      0.000000      0.000000      0.000000             0
           10           50     -0.009891     18.100631      5.262413            57
           15           40     -0.034568     17.211441      7.830370            55
           15           50     -0.040006     17.047938      9.788980            49
           10           40     -0.043763     16.943626      6.075013            65
           20           50     -0.065446     16.153672     10.870249            43
            5           40     -0.076842     15.789201      5.400767            75


## 2. Random Search Optimization

**Random Search** randomly samples parameter combinations.

**Pros:**
- Much faster than grid search
- Works well for high-dimensional spaces
- Can find good solutions quickly

**Cons:**
- No guarantee of finding absolute best
- Results vary between runs (unless fixed seed)

In [7]:
# Define larger parameter space (would take too long for grid search)
large_param_space = {
    'short_window': list(range(5, 21)),      # 16 values
    'long_window': list(range(25, 81)),      # 56 values
    'shares': [10]
}

# Calculate total combinations
total_large = 1
for values in large_param_space.values():
    total_large *= len(values)

print("Random Search Configuration:")
print(f"  Total possible combinations: {total_large:,}")
print(f"  Random samples to test: 100")
print(f"  Coverage: {100/total_large*100:.2f}% of space")

Random Search Configuration:
  Total possible combinations: 896
  Random samples to test: 100
  Coverage: 11.16% of space


In [8]:
# Run random search
random_optimizer = RandomSearchOptimizer(
    n_iter=100,
    random_state=42,
    verbose=True
)

start_time = time.time()
random_results = random_optimizer.optimize(
    data=data,
    config=config,
    strategy_class=MovingAverageStrategy,
    param_space=large_param_space,
    metric='sharpe_ratio'
)
random_time = time.time() - start_time

print(f"\n⏱️  Random search completed in {random_time:.2f} seconds")

Testing 100 random parameter combinations...


Random Search: 100%|██████████| 100/100 [00:15<00:00,  6.38it/s]


⏱️  Random search completed in 15.69 seconds


In [9]:
# Compare best results
print("\nTOP 10 PARAMETER COMBINATIONS (Random Search)")
print("=" * 90)
print(random_results[display_cols].head(10).to_string(index=False))
print("=" * 90)


TOP 10 PARAMETER COMBINATIONS (Random Search)
 short_window  long_window  sharpe_ratio  total_return  max_drawdown  total_trades
           18           27      0.045666     20.001155      5.545254            85
            9           48      0.033390     19.577305      5.615976            55
           10           48      0.008495     18.729923      5.279871            57
           16           78      0.000000      0.000000      0.000000             0
           19           58      0.000000      0.000000      0.000000             0
           11           59      0.000000      0.000000      0.000000             0
            9           67      0.000000      0.000000      0.000000             0
            7           73      0.000000      0.000000      0.000000             0
            5           63      0.000000      0.000000      0.000000             0
           14           78      0.000000      0.000000      0.000000             0


In [10]:
# Compare efficiency
print("\nGRID SEARCH vs RANDOM SEARCH")
print("=" * 60)
print(f"Grid Search:")
print(f"  Combinations tested: {len(grid_results)}")
print(f"  Time: {grid_time:.2f}s")
print(f"  Best Sharpe: {grid_results['sharpe_ratio'].iloc[0]:.4f}")

print(f"\nRandom Search:")
print(f"  Combinations tested: {len(random_results)}")
print(f"  Time: {random_time:.2f}s")
print(f"  Best Sharpe: {random_results['sharpe_ratio'].iloc[0]:.4f}")
print(f"  Speedup: {(len(grid_results)/len(random_results)):.1f}x fewer tests")
print("=" * 60)


GRID SEARCH vs RANDOM SEARCH
Grid Search:
  Combinations tested: 16
  Time: 2.84s
  Best Sharpe: 0.0000

Random Search:
  Combinations tested: 100
  Time: 15.69s
  Best Sharpe: 0.0457
  Speedup: 0.2x fewer tests


## 3. Walk-Forward Optimization

**Walk-Forward** splits data into train/test to prevent overfitting.

**Key Principle**: Optimize on past data (train), validate on future data (test).

**Why It's Important:**
- Parameters optimized on full dataset are often overfit
- Walk-forward shows realistic out-of-sample performance
- Industry standard for robust optimization

In [11]:
# Create walk-forward optimizer
wfo = WalkForwardOptimizer(
    train_size=0.7,  # 70% for training, 30% for testing
    verbose=True
)

# Use smaller parameter space for demonstration
wf_param_space = {
    'short_window': [5, 10, 15, 20],
    'long_window': [30, 40, 50, 60],
    'shares': [10]
}

print("Walk-Forward Optimization Configuration:")
print(f"  Train/Test Split: {wfo.train_size*100:.0f}% / {(1-wfo.train_size)*100:.0f}%")
print(f"  Optimization Metric: sharpe_ratio")

Walk-Forward Optimization Configuration:
  Train/Test Split: 70% / 30%
  Optimization Metric: sharpe_ratio


In [12]:
# Run walk-forward optimization
wf_results = wfo.optimize(
    data=data,
    config=config,
    strategy_class=MovingAverageStrategy,
    param_space=wf_param_space,
    metric='sharpe_ratio'
)


Walk-Forward Optimization
Train period: 2015-01-02 00:00:00 to 2021-04-19 00:00:00
  - Rows: 1584
Test period: 2021-04-20 00:00:00 to 2023-12-29 00:00:00
  - Rows: 680

Phase 1: Optimizing on training data...
Testing 16 parameter combinations...


Grid Search: 100%|██████████| 16/16 [00:02<00:00,  7.96it/s]



Phase 2: Testing 16 combinations on test data...


Running strategies: 100%|██████████| 1/1 [00:00<00:00, 31.92it/s]


Strategy MA_20.0_50.0 error at 2021-06-30 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-20.0] of type float64
Strategy MA_20.0_50.0 error at 2021-07-01 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-20.0] of type float64
Strategy MA_20.0_50.0 error at 2021-07-02 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-20.0] of type float64
Strategy MA_20.0_50.0 error at 2021-07-06 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-20.0] of type float64
Strategy MA_20.0_50.0 error at 2021-07-07 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-20.0] of type float64
Strategy MA_20.0_50.0 error at 2021-07-08 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-20.0] of type float64
Strategy MA_20.0_50.0 error at 2021-07-09 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-20.0] of type float64

...

Strategy MA_5.0_40.0 error at 2021-06-30 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_40.0 error at 2021-07-01 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_40.0 error at 2021-07-02 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_40.0 error at 2021-07-06 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_40.0 error at 2021-07-07 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_40.0 error at 2021-07-08 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_40.0 error at 2021-07-09 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5

Running strategies:   0%|          | 0/1 [00:00<?, ?it/s]

Strategy MA_5.0_50.0 error at 2021-06-30 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_50.0 error at 2021-07-01 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_50.0 error at 2021-07-02 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_50.0 error at 2021-07-06 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_50.0 error at 2021-07-07 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_50.0 error at 2021-07-08 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5.0_50.0 error at 2021-07-09 00:00:00: cannot do positional indexing on DatetimeIndex with these indexers [-5.0] of type float64
Strategy MA_5

Running strategies: 100%|██████████| 1/1 [00:00<00:00, 30.68it/s]


Walk-Forward Optimization Complete

Best parameters (by test sharpe_ratio):
  short_window: 20.0
  long_window: 50.0
  shares: 10.0

Performance:
  Train sharpe_ratio: 0.3409
  Test sharpe_ratio: 0.0000
  Difference: -0.3409



In [13]:
# Analyze train vs test performance
print("\nWALK-FORWARD RESULTS (Top 10 by Test Performance)")
print("=" * 100)
wf_display_cols = ['short_window', 'long_window', 'train_sharpe_ratio', 'test_sharpe_ratio', 
                   'sharpe_ratio_diff', 'train_total_return', 'test_total_return']
print(wf_results[wf_display_cols].head(10).to_string(index=False))
print("=" * 100)


WALK-FORWARD RESULTS (Top 10 by Test Performance)
 short_window  long_window  train_sharpe_ratio  test_sharpe_ratio  sharpe_ratio_diff  train_total_return  test_total_return
           20           50            0.340869                0.0          -0.340869           19.458186                0.0
           15           50            0.315196                0.0          -0.315196           18.935072                0.0
           20           40            0.225676                0.0          -0.225676           17.198187                0.0
           15           40            0.109950                0.0          -0.109950           14.752313                0.0
           20           30            0.061059                0.0          -0.061059           13.728483                0.0
           15           30            0.046791                0.0          -0.046791           13.439507                0.0
            5           60            0.000000                0.0           0.000

In [14]:
# Check for overfitting
print("\nOVERFITTING ANALYSIS:")
print("=" * 60)

# Calculate average train/test gap
avg_train_sharpe = wf_results['train_sharpe_ratio'].mean()
avg_test_sharpe = wf_results['test_sharpe_ratio'].mean()
degradation = (avg_train_sharpe - avg_test_sharpe) / avg_train_sharpe * 100

print(f"Average Train Sharpe: {avg_train_sharpe:.4f}")
print(f"Average Test Sharpe:  {avg_test_sharpe:.4f}")
print(f"Performance Degradation: {degradation:.2f}%")

print(f"\nInterpretation:")
if degradation < 10:
    print("  ✅ Low degradation - parameters generalize well")
elif degradation < 30:
    print("  ⚠️  Moderate degradation - some overfitting present")
else:
    print("  ❌ High degradation - significant overfitting!")
    
print("=" * 60)


OVERFITTING ANALYSIS:
Average Train Sharpe: 0.0265
Average Test Sharpe:  0.0000
Performance Degradation: 100.00%

Interpretation:
  ❌ High degradation - significant overfitting!


## 4. Custom Optimizer

Create your own optimization method by inheriting from `Optimizer` base class.

Let's create two custom optimizers:
1. **Bayesian-Inspired Optimizer**: Focus on promising regions
2. **Multi-Metric Optimizer**: Optimize for multiple metrics simultaneously

In [15]:
class AdaptiveRandomSearchOptimizer(Optimizer):
    """Adaptive random search that focuses on promising parameter regions.
    
    Starts with random search, then focuses on regions with good results.
    """
    
    def __init__(self, n_iter: int = 100, exploration_ratio: float = 0.5, 
                 verbose: bool = True, name: str = None):
        super().__init__(name=name or "AdaptiveRandomSearch")
        self.n_iter = n_iter
        self.exploration_ratio = exploration_ratio
        self.verbose = verbose
    
    def optimize(
        self,
        data: pd.DataFrame,
        config: BacktestConfig,
        strategy_class: Type[Strategy],
        param_space: Dict[str, List[Any]],
        metric: str = "sharpe_ratio",
    ) -> pd.DataFrame:
        """Run adaptive random search."""
        import random
        
        results = []
        param_names = list(param_space.keys())
        
        # Phase 1: Exploration
        n_explore = int(self.n_iter * self.exploration_ratio)
        if self.verbose:
            print(f"Phase 1: Exploring {n_explore} random combinations...")
        
        for i in range(n_explore):
            param_dict = {name: random.choice(param_space[name]) for name in param_names}
            
            try:
                strategy = strategy_class(**param_dict)
                metrics = self._run_backtest(data, config, strategy)
                results.append({**param_dict, **metrics})
            except:
                continue
        
        if not results:
            return pd.DataFrame()
        
        # Phase 2: Exploitation - focus on best region
        n_exploit = self.n_iter - n_explore
        if self.verbose:
            print(f"Phase 2: Exploiting around best results ({n_exploit} iterations)...")
        
        # Find best parameters so far
        results_df = pd.DataFrame(results)
        best_params = results_df.nlargest(3, metric)[param_names].to_dict('records')
        
        for i in range(n_exploit):
            # Sample from best parameters with small variations
            base = random.choice(best_params)
            param_dict = {}
            
            for name in param_names:
                values = param_space[name]
                base_idx = values.index(base[name])
                # Sample nearby values
                nearby_range = max(1, len(values) // 10)
                start = max(0, base_idx - nearby_range)
                end = min(len(values), base_idx + nearby_range + 1)
                param_dict[name] = random.choice(values[start:end])
            
            try:
                strategy = strategy_class(**param_dict)
                metrics = self._run_backtest(data, config, strategy)
                results.append({**param_dict, **metrics})
            except:
                continue
        
        df = pd.DataFrame(results)
        return df.sort_values(metric, ascending=False).reset_index(drop=True)

print("✓ AdaptiveRandomSearchOptimizer created")

✓ AdaptiveRandomSearchOptimizer created


In [16]:
class MultiMetricOptimizer(Optimizer):
    """Optimize for multiple metrics simultaneously using weighted scoring.
    
    Balances multiple objectives (e.g., return, risk, drawdown).
    """
    
    def __init__(self, metric_weights: Dict[str, float] = None, 
                 base_optimizer: Optimizer = None, verbose: bool = True, name: str = None):
        super().__init__(name=name or "MultiMetric")
        # Default weights: prioritize risk-adjusted returns
        self.metric_weights = metric_weights or {
            'sharpe_ratio': 0.4,
            'sortino_ratio': 0.3,
            'total_return': 0.2,
            'max_drawdown': -0.1  # Negative weight (we want to minimize)
        }
        self.base_optimizer = base_optimizer or GridSearchOptimizer(verbose=False)
        self.verbose = verbose
    
    def optimize(
        self,
        data: pd.DataFrame,
        config: BacktestConfig,
        strategy_class: Type[Strategy],
        param_space: Dict[str, List[Any]],
        metric: str = "sharpe_ratio",  # Kept for compatibility
    ) -> pd.DataFrame:
        """Optimize using weighted combination of metrics."""
        if self.verbose:
            print("Multi-Metric Optimization")
            print("Metric weights:")
            for m, w in self.metric_weights.items():
                print(f"  {m}: {w:.2f}")
        
        # Run base optimization
        results = self.base_optimizer.optimize(
            data, config, strategy_class, param_space, metric
        )
        
        if results.empty:
            return results
        
        # Normalize each metric to [0, 1]
        for metric_name in self.metric_weights.keys():
            if metric_name in results.columns:
                col = results[metric_name]
                min_val = col.min()
                max_val = col.max()
                if max_val != min_val:
                    results[f"{metric_name}_norm"] = (col - min_val) / (max_val - min_val)
                else:
                    results[f"{metric_name}_norm"] = 0.5
        
        # Calculate weighted score
        results['weighted_score'] = 0
        for metric_name, weight in self.metric_weights.items():
            if f"{metric_name}_norm" in results.columns:
                results['weighted_score'] += results[f"{metric_name}_norm"] * weight
        
        # Sort by weighted score
        results = results.sort_values('weighted_score', ascending=False).reset_index(drop=True)
        
        return results

print("✓ MultiMetricOptimizer created")

✓ MultiMetricOptimizer created


### Test Custom Optimizers

In [17]:
# Test adaptive random search
adaptive_optimizer = AdaptiveRandomSearchOptimizer(n_iter=50, exploration_ratio=0.4, verbose=True)

adaptive_results = adaptive_optimizer.optimize(
    data=data,
    config=config,
    strategy_class=MovingAverageStrategy,
    param_space=large_param_space,
    metric='sharpe_ratio'
)

print("\nTOP 5 RESULTS (Adaptive Random Search):")
print("=" * 90)
print(adaptive_results[display_cols].head(5).to_string(index=False))

Phase 1: Exploring 20 random combinations...


Running strategies: 100%|██████████| 1/1 [00:00<00:00,  6.26it/s]


Phase 2: Exploiting around best results (30 iterations)...


Running strategies: 100%|██████████| 1/1 [00:00<00:00, 13.53it/s]


TOP 5 RESULTS (Adaptive Random Search):
 short_window  long_window  sharpe_ratio  total_return  max_drawdown  total_trades
            7           59           0.0           0.0           0.0             0
            8           55           0.0           0.0           0.0             0
           14           63           0.0           0.0           0.0             0
            6           64           0.0           0.0           0.0             0
           14           63           0.0           0.0           0.0             0


In [18]:
# Test multi-metric optimizer
multi_optimizer = MultiMetricOptimizer(
    metric_weights={
        'sharpe_ratio': 0.35,
        'sortino_ratio': 0.25,
        'total_return': 0.20,
        'max_drawdown': -0.15,  # Minimize drawdown
        'win_rate': 0.05
    },
    verbose=True
)

multi_results = multi_optimizer.optimize(
    data=data,
    config=config,
    strategy_class=MovingAverageStrategy,
    param_space=wf_param_space,
    metric='sharpe_ratio'
)

print("\nTOP 5 RESULTS (Multi-Metric Optimizer):")
print("=" * 100)
multi_display_cols = ['short_window', 'long_window', 'weighted_score', 'sharpe_ratio', 
                      'sortino_ratio', 'total_return', 'max_drawdown']
print(multi_results[multi_display_cols].head(5).to_string(index=False))

Multi-Metric Optimization
Metric weights:
  sharpe_ratio: 0.35
  sortino_ratio: 0.25
  total_return: 0.20
  max_drawdown: -0.15
  win_rate: 0.05


Running strategies: 100%|██████████| 1/1 [00:00<00:00, 13.45it/s]


TOP 5 RESULTS (Multi-Metric Optimizer):
 short_window  long_window  weighted_score  sharpe_ratio  sortino_ratio  total_return  max_drawdown
           10           50        0.744460     -0.009891      -0.011970     18.100631      5.262413
           15           40        0.638686     -0.034568      -0.041661     17.211441      7.830370
           10           40        0.625209     -0.043763      -0.052962     16.943626      6.075013
            5           60        0.600000      0.000000       0.000000      0.000000      0.000000
           10           60        0.600000      0.000000       0.000000      0.000000      0.000000


## 5. Optimization Best Practices

### Common Pitfalls and How to Avoid Them

### Pitfall #1: Overfitting

**Problem**: Parameters work great on historical data but fail on new data.

**Solutions:**
- Use walk-forward optimization
- Hold out test data
- Simplify strategies (fewer parameters)
- Add regularization (penalty for extreme values)

In [19]:
# Demonstrate overfitting with too many parameters
print("OVERFITTING DEMONSTRATION")
print("=" * 60)
print("\nRule of thumb: More parameters = higher overfitting risk")
print("\nParameter count vs overfitting risk:")
print("  2-3 parameters:  Low risk")
print("  4-6 parameters:  Moderate risk (use walk-forward)")
print("  7+ parameters:   High risk (definitely use walk-forward)")
print("\nData requirements:")
print("  Minimum: 500-1000 data points per parameter")
print(f"  Your data: {len(data)} points")
print(f"  Can safely optimize: {len(data) // 500} parameters")
print("=" * 60)

OVERFITTING DEMONSTRATION

Rule of thumb: More parameters = higher overfitting risk

Parameter count vs overfitting risk:
  2-3 parameters:  Low risk
  4-6 parameters:  Moderate risk (use walk-forward)
  7+ parameters:   High risk (definitely use walk-forward)

Data requirements:
  Minimum: 500-1000 data points per parameter
  Your data: 2264 points
  Can safely optimize: 4 parameters


### Pitfall #2: Look-Ahead Bias

**Problem**: Using future information in optimization.

**Solutions:**
- Always optimize on past data only
- Use proper train/test splits
- Never include test period in optimization

### Pitfall #3: Data Snooping

**Problem**: Running many optimizations and picking the best result.

**Solutions:**
- Document all optimization attempts
- Use consistent validation methodology
- Be skeptical of "too good" results

### Pitfall #4: Optimizing the Wrong Metric

**Problem**: Maximizing return without considering risk.

**Solutions:**
- Use risk-adjusted metrics (Sharpe, Sortino)
- Consider drawdown limits
- Multi-objective optimization

In [20]:
# Compare optimizing different metrics
print("METRIC SELECTION COMPARISON")
print("=" * 80)

metrics_to_compare = ['total_return', 'sharpe_ratio', 'sortino_ratio']

for opt_metric in metrics_to_compare:
    opt = GridSearchOptimizer(verbose=False)
    results = opt.optimize(
        data=data,
        config=config,
        strategy_class=MovingAverageStrategy,
        param_space={'short_window': [5, 10, 15], 'long_window': [30, 40, 50], 'shares': [10]},
        metric=opt_metric
    )
    
    best = results.iloc[0]
    print(f"\nOptimizing for {opt_metric}:")
    print(f"  Best params: short={best['short_window']}, long={best['long_window']}")
    print(f"  Total Return: {best['total_return']*100:.2f}%")
    print(f"  Sharpe Ratio: {best['sharpe_ratio']:.2f}")
    print(f"  Max Drawdown: {best['max_drawdown']*100:.2f}%")

print("\n" + "="*80)
print("Note: Different metrics lead to different 'optimal' parameters!")
print("Choose metric that matches your risk tolerance and goals.")
print("="*80)

METRIC SELECTION COMPARISON


Running strategies: 100%|██████████| 1/1 [00:00<00:00,  6.14it/s]



Optimizing for total_return:
  Best params: short=10.0, long=50.0
  Total Return: 1810.06%
  Sharpe Ratio: -0.01
  Max Drawdown: 526.24%


Running strategies: 100%|██████████| 1/1 [00:00<00:00,  6.14it/s]



Optimizing for sharpe_ratio:
  Best params: short=10.0, long=50.0
  Total Return: 1810.06%
  Sharpe Ratio: -0.01
  Max Drawdown: 526.24%


Running strategies: 100%|██████████| 1/1 [00:00<00:00,  6.09it/s]


Optimizing for sortino_ratio:
  Best params: short=10.0, long=50.0
  Total Return: 1810.06%
  Sharpe Ratio: -0.01
  Max Drawdown: 526.24%

Note: Different metrics lead to different 'optimal' parameters!
Choose metric that matches your risk tolerance and goals.


## Summary

In this notebook, we explored advanced optimization:

1. ✅ **Grid Search**: Exhaustive but slow - best for small spaces
2. ✅ **Random Search**: Fast and effective - good for large spaces
3. ✅ **Walk-Forward**: Prevents overfitting - industry standard
4. ✅ **Custom Optimizers**: Created adaptive and multi-metric optimizers
5. ✅ **Best Practices**: Learned how to avoid common pitfalls

### Key Takeaways:

**Choosing an Optimizer:**
- **Small space (< 50 combos)**: Grid Search
- **Large space**: Random Search or Adaptive
- **Production strategies**: Always use Walk-Forward
- **Multiple objectives**: Multi-Metric Optimizer

**Optimization Checklist:**
- ✅ Use walk-forward or cross-validation
- ✅ Hold out test data
- ✅ Optimize risk-adjusted metrics
- ✅ Limit number of parameters
- ✅ Have enough data (500-1000 points per parameter)
- ✅ Check for train/test performance gap
- ✅ Document all optimization attempts
- ✅ Be skeptical of perfect results

### Creating Custom Optimizers:

```python
from simple_backtest.optimization import Optimizer

class MyOptimizer(Optimizer):
    def optimize(self, data, config, strategy_class, param_space, metric):
        # Your optimization logic
        # Use self._run_backtest(data, config, strategy) helper
        return results_dataframe
```

### Advanced Techniques Not Covered:

- **Genetic Algorithms**: Evolutionary optimization
- **Bayesian Optimization**: Smart sampling using past results
- **Cross-Validation**: Multiple train/test splits
- **Combinatorial Optimization**: For discrete parameter spaces
- **Online Learning**: Continuous parameter adaptation

### Remember:

> "The best strategy is not the one with the highest backtest return,
> but the one that performs consistently out-of-sample."

### Final Warning:

**Past performance does not guarantee future results!**

Even with perfect optimization:
- Markets change
- Strategies decay
- Transaction costs matter
- Live trading has slippage

Always paper trade optimized strategies before risking real money!